In [ ]:
from qiskit.tools.jupyter import *
from qiskit import IBMQ
IBMQ.save_account('WRITE YOUR OWN ACCOUNT',
                     overwrite=True)
provider = IBMQ.load_account()

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

backend = BasicAer.get_backend('unitary_simulator')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.u3(pi/2,pi/2,pi/2,q)
qc.draw(output='mpl')

In [ ]:
job = execute(qc, backend)
job.result().get_unitary(qc, decimals=3)

In [ ]:
from qiskit.tools.jupyter import *
from qiskit import IBMQ

from qiskit import QuantumRegister, ClassicalRegister 
from qiskit import QuantumCircuit, Aer
from qiskit import execute

# Create one 4 qubit QuantumRegister to hold the Bayesian network and an ancilla qubit,
# and a 3 bit ClassicalRegister to hold the sampled values
net = QuantumRegister(4, 'qreg')
cl = ClassicalRegister(3, 'creg')

circ = QuantumCircuit(net, cl, name='circ')

In [ ]:
from numpy import arcsin, sqrt, pi

def probToAngle(prob):
    """
    Converts a given P(1) value into an equivalent theta value.
    """
    return 2*arcsin(sqrt(prob))

In [ ]:
# Setting up a qubit to represent the variable P
circ.u3(probToAngle(0.35), 0, 0, net[0])

# Since we have P = 1, we use the second row of the probability table for the variable E
circ.u3(probToAngle(0.76), 0, 0, net[1])

# Setting up the qubit representing H assuming that E = 0
circ.u3(probToAngle(0.39), 0, 0, net[2])

In [ ]:
def oracle(circ):
    """
    Implements an oracle that flips the sign of states that contain P = 1.
    """
    circ.cu3(pi, pi, 0, net[0], net[1])
    circ.cu3(pi, pi, 0, net[0], net[1])    
    return circ


def u_gate(circ):
    """
    Implements the U gate that flips states about the average amplitude.
    """
    # Implements the quantum circuit that converts ψ -> |000...0>
    circ.u3(-1*probToAngle(0.35), 0, 0, net[0])
    circ.u3(-1*probToAngle(0.76), 0, 0, net[1])
    circ.u3(-1*probToAngle(0.39), 0, 0, net[2])

    # Flipping the |000...0> state using a triple controlled Z gate condtioned on P, E and H, 
    # and applied to the ancilla
    circ.x(net)
    circ.cu1(pi/4, net[0], net[3])
    circ.cx(net[0], net[1])
    circ.cu1(-pi/4, net[1], net[3])
    circ.cx(net[0], net[1])
    circ.cu1(pi/4, net[1], net[3])
    circ.cx(net[1], net[2])
    circ.cu1(-pi/4, net[2], net[3])
    circ.cx(net[0], net[2])
    circ.cu1(pi/4, net[2], net[3])
    circ.cx(net[1], net[2])
    circ.cu1(-pi/4, net[2], net[3])
    circ.cx(net[0], net[2])
    circ.cu1(pi/4, net[2], net[3])
    circ.x(net)

    # Implements the quantum circuit that converts |000...0> -> ψ 
    circ.u3(probToAngle(0.35), 0, 0, net[0])
    circ.u3(probToAngle(0.76), 0, 0, net[1])
    circ.u3(probToAngle(0.39), 0, 0, net[2])

    return circ

In [ ]:
# Apply oracle and U gate twice
circ = oracle(circ)
circ = u_gate(circ)
circ = oracle(circ)
circ = u_gate(circ)
circ.x(net[0])

# Measure E, and rotate H to the P(1) value in the second row of the P(H|E) table condtioned on E
circ.measure(net[1], cl[1])
circ.u3(probToAngle(0.82) - probToAngle(0.39), 0, 0, net[2])

# Sample by measuring the rest of the qubits
circ.measure(net[0], cl[0])
circ.measure(net[2], cl[2])

In [ ]:
import pandas as pd
data=pd.read_csv('ch_price.csv')

In [ ]:
import numpy as np
price_data=data['price']
price_data=price_data.values
price_data=price_data.tolist()

In [ ]:
samples_list = []
samples_list.append(price_data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *
from qiskit.aqua.algorithms import Grover
from sklearn.preprocessing import MinMaxScaler

# First princinple for two parent nodes and one child
class byskit():
    def __init__(self, backend, parents, child, evd = None):
        self.backend = backend
        self.parents = parents
        self.child = child
        self.n = int(np.shape(parents)[0])
        self.n_child = np.shape(child)[1]
        self.ctrl = QuantumRegister(self.n, 'ctrl')
        self.anc = QuantumRegister(self.n - 1, 'anc')
        self.tgt = QuantumRegister(self.n_child, 'tgt')
        if evd != None:
            self.oracle = QuantumRegister(evd,'oracle')
            self.circ = QuantumCircuit(self.ctrl, self.anc, self.tgt, self.oracle)
        else:
            self.circ = QuantumCircuit(self.ctrl, self.anc, self.tgt)

        #self.c_ctrl = ClassicalRegister(self.n, 'c_ctrl')
        #self.c_tgt = ClassicalRegister(self.n_child, 'c_tgt')

        self.parent_init()
        self.child_init()

    def parent_init(self):
        for i in range(self.n):
            theta = self.calc_theta(self.parents[0], self.parents[1])
            #theta = self.calc_theta(self.parents[2*i], self.parents[2*i+1])
            self.circ.ry(theta, i)

        self.circ.barrier()

    def child_init(self):
        self.a = np.arange(0, 2 ** self.n)
        self.gates = []
        for i in self.a:
            s = str(np.binary_repr(i, width=self.n))
            self.gates.append(s)

        for i in range(2**self.n):
            self.xgate(self.gates[i])
            for j in range(self.n_child):
                #theta = self.calc_theta(self.child[2 * i + 1,j], self.child[2 * i,j])
                theta = self.calc_theta(self.child[0,j], self.child[1,j])
                self.cn_ry(theta,j)
            self.xgate(self.gates[i])
            self.circ.barrier()

    def xgate(self,gate):
        for index, item in enumerate(gate):
            if int(item) == 0:
                self.circ.x(index)

    #RY gates
    def cn_ry(self,theta,target):
        # compute
        self.circ.ccx(self.ctrl[0], self.ctrl[1], self.anc[0])
        for i in range(2, self.n):
            self.circ.ccx(self.ctrl[i], self.anc[i - 2], self.anc[i - 1])

        # copy
        self.circ.cry(theta,self.anc[self.n - 2], self.tgt[target])

        # uncompute
        for i in range(self.n - 1, 1, -1):
            self.circ.ccx(self.ctrl[i], self.anc[i - 2], self.anc[i - 1])
        self.circ.ccx(self.ctrl[0], self.ctrl[1], self.anc[0])


    def calc_theta(self,p1,p0):
        return 2 * np.arctan(np.sqrt((p1)/(p0)))

    def plot(self):
        self.circ.draw()
        plt.show()

    def execute_circ(self):
        self.circ.measure_all()
        results = execute(self.circ, self.backend, shots=100)
        return results


    def evaluate(self, samples_list, observations):
        p_o = 0
        for sample in samples_list:
            accept = True
            for o in observations:
                if sample[observations[o]['n']] == observations[o]['state']:
                    pass
                else:
                    accept = False
            if accept == True:
                #print('Observation true given evidence')
                p_o += 1
        p_o /= len(samples_list)

        return p_o

    def amplitude_amplification(self,evidence):
        self.state_preparation = self.circ
        self.oracle = QuantumCircuit(self.ctrl, self.anc, self.tgt)
        for index, e in enumerate(evidence):
            if evidence[e]['state'] == '1':
                self.oracle.z([evidence[e]['n']])

        self.grover_op = Grover(self.oracle, state_preparation=self.state_preparation)
        self.grover_op.draw()

    def oracle(self):
        pass

    def u_gate(self):
        pass

def gen_random_weights(n_parent,n_child):
    parents = []
    parents.append(samples_list[0][1])
    parents.append(samples_list[0][2])
    parents = np.array(parents)

    child = np.random.rand(2 ** (n_parent + 1), n_child)
    for i in range(n_child):
        for j in range(2 ** (n_parent)):
            child[2 * j + 1, i] = 1 - child[2 * j, i]
            

    return parents, child

if __name__=='__main__':
    from qiskit import IBMQ
    IBMQ.save_account('WRITE YOUR OWN ACCOUNT',
                     overwrite=True)
    provider = IBMQ.load_account()
    backend = provider.get_backend('ibmq_qasm_simulator')
    #provider = IBMQ.get_provider(hub='ibm-q-oxford', group='on-boarding', project='on-boarding-proj')
    from qiskit import Aer
    #backend = BasicAer.get_backend('unitary_simulator')
    #backend = Aer.get_backend('qasm_simulator')

    n_parent = 2
    n_child = 1

    parents, children = gen_random_weights(n_parent, n_child)
    b = byskit(backend, parents, children)
    b.plot()

    evidence = {
        'one':{
            'n':1,
            'state':'0'
        }
    }

    observations = {
        'three':{
            'n':1,
            'state':'0'
        }
    }

    prob = b.evaluate(samples_list, observations)

In [ ]:
#from byskit import byskit,gen_random_weights
parents,children = gen_random_weights(n_parent,n_child)
b = byskit(backend,parents,children)
b.circ.draw(output='mpl')

In [ ]:
results = b.execute_circ().result()

In [ ]:
from qiskit.tools.visualization import plot_histogram

plot_histogram(results.get_counts(b.circ))

In [ ]:
print(results.get_counts(b.circ))

In [ ]:
import operator
circ={}
circ=results.get_counts(b.circ)

a=max(circ.items(), key=operator.itemgetter(1))[0]
b=max(circ.items(), key=operator.itemgetter(1))[1]
print(a)
print(b)

In [ ]:
if a=="0000":
    print("내일의 무 가격은 20이하로 예측됩니다.")
    pred="10"
if a=="0001":
    print("내일의 무 가격은 21~40으로 예측됩니다.")
    pred="30"
if a=="0010":
    print("내일의 무 가격은 41~60으로 예측됩니다.")
    pred="50"
if a=="0011":
    print("내일의 무 가격은 61~80으로 예측됩니다.")
    pred="70"
if a=="1000":
    print("내일의 무 가격은 81~100으로 예측됩니다.")
    pred="90"
if a=="1001":
    print("내일의 무 가격은 101~120으로 예측됩니다.")
    pred="110"
if a=="1010":
    print("내일의 무 가격은 121~140으로 예측됩니다.")
    pred="130"
if a=="1011":
    print("내일의 무 가격은 141이상으로 예측됩니다.")
    pred="150"
pred=int(pred)

In [ ]:
from matplotlib import pyplot as plt

x_values = [1,2,3]
y_values = [parents[0], parents[1], pred]
plt.plot(x_values, y_values)
plt.show()

In [ ]:
phase_1=(parents[1]-parents[0])/parents[0]*100+100
phase_2=(pred-parents[1])/parents[1]*100+100

if (phase_1>=90)and(phase_1<=140):
    if (phase_2>=20)and(phase_2<=90):
        print("up down up down pattern")
        print("이틀 후 증가할 것입니다.")
        
if (phase_1>=20)and(phase_1<=90):
    if (phase_2>=110)and(phase_2<=180):
        print("up down up down pattern")
        print("이틀 후 감소할 것입니다.")
        
if (phase_1>=110)and(phase_1<=180):
    if (phase_2>=20)and(phase_2<=90):
        print("up down up down pattern")
        print("이틀 후 증가할 것입니다.")
        
if (phase_1>=85)and(phase_1<=90):
    if (phase_2>=100)and(phase_2<=155):
        print("big spike pattern")
        print("이틀 후 증가할 것입니다.")
        
if (phase_1>=100)and(phase_1<=155):
    if (phase_2>=160)and(phase_2<=610):
        print("big spike pattern")
        print("이틀 후 감소할 것입니다.")
        
if (phase_1<=90):
    if (phase_2<=50):
        print("big spike pattern")
        print("이틀 후 감소할 것입니다.")
        
if (phase_1>=85)and(phase_1<=90):
    if (phase_2>=85)and(phase_2<=90):
        print("decreasing pattern")
        print("이틀 후 감소할 것입니다.")
        
if (phase_1>=40)and(phase_1<=90):
    if (phase_2>=100)and(phase_2<=200):
        print("small spike pattern")
        print("이틀 후 증가할 것입니다.")
        
if (phase_1>=100)and(phase_1<=200):
    if (phase_2>=100)and(phase_2<=210):
        print("small spike pattern")
        print("이틀 후 감소할 것입니다.")
        
else:
    print("패턴을 예측할 수 없습니다.")